# 5.5 卷积神经网络（LeNet）

在3.9节（多层感知机的从零开始实现）里我们构造了一个含单隐藏层的多层感知机模型来对Fashion-MNIST数据集中的图像进行分类。每张图像高和宽均是28像素。我们将图像中的像素逐行展开，得到长度为784的向量，并输入进全连接层中。然而，这种分类方法有一定的局限性。

1. 图像在同一列邻近的像素在这个向量中可能相距较远。它们构成的模式可能难以被模型识别。
2. 对于大尺寸的输入图像，使用全连接层容易造成模型过大。假设输入是高和宽均为1000像素的彩色照片（含3个通道）。即使全连接层输出个数仍是256，该层权重参数的形状是$3,000,000\times 256$：它占用了大约3 GB的内存或显存。这带来过复杂的模型和过高的存储开销。

卷积层尝试解决这两个问题。一方面，卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别；另一方面，卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

卷积神经网络就是含卷积层的网络。本节里我们将介绍一个早期用来识别手写数字图像的卷积神经网络：LeNet [1]。这个名字来源于LeNet论文的第一作者Yann LeCun。LeNet展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。这个奠基性的工作第一次将卷积神经网络推上舞台，为世人所知。LeNet的网络结构如下图所示。

## 5.5.1 LeNet模型

LeNet分为卷积层块和全连接层块两个部分。下面我们分别介绍这两个模块。

卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用$5\times 5$的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为$2\times 2$，且步幅为2。由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠。

卷积层块的输出形状为(批量大小, 通道, 高, 宽)。当卷积层块的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。

下面我们通过`Sequential`类来实现LeNet模型。

In [1]:
import time
import torch
from torch import nn, optim


import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
print(torch.__version__)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/mylady/anaconda3/envs/d2l_pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.1.2+cu121


In [2]:
device

device(type='cpu')

In [3]:

class LeNet(nn.Module):
    
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output


接下来查看每个层的形状。

In [4]:
net = LeNet()

print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


可以看到，在卷积层块中输入的高和宽在逐层减小。卷积层由于使用高和宽均为5的卷积核，从而将高和宽分别减小4，而池化层则将高和宽减半，但通道数则从1增加到16。全连接层则逐层减少输出个数，直到变成图像的类别数10。

## 5.5.2 获取数据和训练模型

下面我们来实验LeNet模型。实验中，我们仍然使用Fashion-MNIST作为训练数据集。

In [5]:
batch_size = 256


train_iter, test_iter = d2l.load_data_fashion_mnist(
    batch_size=batch_size,
    root="/home/mylady/ai_data/d2l_data/FashionMNIST"
)

因为卷积神经网络计算比多层感知机要复杂，建议使用GPU来加速计算。因此，我们对3.6节（softmax回归的从零开始实现）中描述的`evaluate_accuracy`函数略作修改，使其支持GPU计算。

In [6]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用。该函数将被逐步改进。
def evaluate_accuracy(data_iter, net, device=None):
    
    # 如果没指定device就使用net的device
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device

    acc_sum = 0.0
    n = 0
    
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                # 评估模式, 这会关闭dropout
                net.eval() 
                
                # 转移到GPU运算
                y_hat = net(X.to(device)).argmax(dim=1) == y.to(device)
                
                # 转移到CPU相加
                acc_sum += (y_hat).float().sum().cpu().item()
                
                # 改回训练模式
                net.train() 

            # 自定义的模型, 3.13节之后不会用到, 不考虑GPU    
            else: 
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    
                    res_1 = net(X, is_training=False).argmax(dim=1) == y
                    acc_sum += (res_1).float().sum().item() 
                else:
                    res_1 = net(X).argmax(dim=1) == y
                    acc_sum += (res_1).float().sum().item() 
            
            n += y.shape[0]
            pass
        pass

    return acc_sum / n

我们同样对3.6节中定义的`train_ch3`函数略作修改，确保计算使用的数据和模型同在内存或显存上。

In [7]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_ch5(net, train_iter, test_iter, 
              batch_size, 
              optimizer, 
              device, num_epochs):
    
    net = net.to(device)
    print("training on ", device)
    
    loss = torch.nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        
        train_l_sum = 0.0
        train_acc_sum = 0.0
        n = 0
        batch_count = 0 
        start = time.time()
        
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            
            train_l_sum += l.cpu().item()
            
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
            pass
        test_acc = evaluate_accuracy(test_iter, net)
        
        epoch_count = epoch + 1             # 训练批次
        loss_cp = train_l_sum / batch_count # 损失计算
        train_acc = train_acc_sum / n       # 正确率
        time_consume = time.time() - start  # 耗时
        
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time_ %.1f sec'
              % (epoch_count, loss_cp, train_acc, test_acc, time_consume)
             )

学习率采用0.001，训练算法使用Adam算法，损失函数使用交叉熵损失函数。

In [8]:
lr = 0.001
num_epochs = 20


optimizer = torch.optim.Adam(net.parameters(), 
                             lr=lr)

In [9]:
# 开始训练


train_ch5(net, train_iter, test_iter, 
          batch_size, optimizer, 
          device, 
          num_epochs
         )

training on  cpu
epoch 1, loss 1.8387, train acc 0.328, test acc 0.596, time_ 2.4 sec
epoch 2, loss 0.9490, train acc 0.635, test acc 0.690, time_ 2.3 sec
epoch 3, loss 0.7877, train acc 0.709, test acc 0.720, time_ 2.5 sec
epoch 4, loss 0.7023, train acc 0.735, test acc 0.736, time_ 2.4 sec
epoch 5, loss 0.6434, train acc 0.751, test acc 0.753, time_ 2.3 sec
epoch 6, loss 0.6004, train acc 0.765, test acc 0.762, time_ 2.2 sec
epoch 7, loss 0.5684, train acc 0.777, test acc 0.776, time_ 2.4 sec
epoch 8, loss 0.5396, train acc 0.788, test acc 0.789, time_ 2.2 sec
epoch 9, loss 0.5145, train acc 0.800, test acc 0.797, time_ 2.3 sec
epoch 10, loss 0.4929, train acc 0.813, test acc 0.800, time_ 2.2 sec
epoch 11, loss 0.4746, train acc 0.822, test acc 0.814, time_ 2.1 sec
epoch 12, loss 0.4610, train acc 0.828, test acc 0.819, time_ 2.1 sec
epoch 13, loss 0.4492, train acc 0.834, test acc 0.826, time_ 2.1 sec
epoch 14, loss 0.4385, train acc 0.839, test acc 0.831, time_ 2.2 sec
epoch 15, lo

In [10]:
# 保存模型
import datetime

str_time = str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

# 模型保存
torch.save(net, 'leNet_%s.pt' % str_time)  # 全保存 39M

## 小结

* 卷积神经网络就是含卷积层的网络。
* LeNet交替使用卷积层和最大池化层后接全连接层来进行图像分类。

In [12]:
# 加载模型

# 模型加载
# model = LeNet()
model = torch.load('leNet_2023-06-26_22-33-56.pt')

model

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [11]:



test_acc = evaluate_accuracy(test_iter, net)

In [12]:
test_acc

0.8485